# Embedding training and testing data sets

In [1]:
%reset -f
import logging
import sys

my_stderr = sys.stderr = open('errors.txt', 'w')  # redirect stderr to file
get_ipython().log.handlers[0].stream = my_stderr  # log errors to new stderr
get_ipython().log.setLevel(logging.INFO)  # errors are logged at info level

import os
import sys
import argparse
import ipdb
from tqdm import tqdm
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import tensorflow_text

2022-05-17 09:23:30.044141: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# LASER="LASER-master"
mldir = "ml-training/SECRECY-EN/data" # training data and embeddings are held here
model_name = "SECRECY-EN"
edir = mldir + "/embed_simplified" # location of laser script that that embeds to LASER SS
#embedding_name = 'USE'
embedding_name = 'USEm'

In [4]:
def USE(txt_list, use_embedder):
    output = []
    txt_df = pd.Series(txt_list)
    batch_size = 124
    for k,g in tqdm(txt_df.groupby(np.arange(len(txt_df))//batch_size)):
        output.append(use_embedder(g.tolist()).numpy())
    output = np.concatenate(output, axis=0).flatten()
    print(output.shape)
    print(output.min())
    print(output.max())
    print('\n')
    return output

In [5]:
if not os.path.exists(edir):
    os.mkdir(edir)

#lang="en"
for lang in ['en','es']:
    print('\nProcessing:')
    #embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

    for part in ('train', 'dev', 'test'):
        # Spanish only exists for test
        if (lang=='es') & (part!='test'):
            continue
        cfname = os.path.join(mldir, part)
        if os.path.exists(cfname + '.' + embedding_name + '.enc.' + lang): # embedding_name
            print(cfname + '.' + embedding_name + '.enc.' + lang + ' Exists')
        txt = []
        
        # copy original files to embed directory, then embed it
        os.system('cp ' + cfname + '.txt.' + lang + ' ' + os.path.join(edir, part) + '.txt.' + lang)
        os.system('cp ' + cfname + '.lbl.' + lang + ' ' + os.path.join(edir, part) + '.lbl.' + lang)
        
        # embed file in embed directory
        with open(cfname + '.txt.' + lang) as f:
            for l in f:
                txt.append(l)
        use_np = USE(txt, embed)
        print(use_np.dtype, use_np.shape)
        use_np.tofile(edir + "/" + part + '.' + embedding_name + '.enc.' + lang)


Processing:
(5427200,)
-0.24683398
0.2476417


float32 (5427200,)
(1357312,)
-0.22874442
0.2429925


float32 (1357312,)
(17886208,)
-0.23657498
0.26058263


float32 (17886208,)

Processing:
(2135040,)
-0.25308377
0.29279864


float32 (2135040,)
